http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf

In [11]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import time
import cmath
import sys
import random
from mnist import MNIST

In [12]:
def loadMNIST():
    mndata = MNIST('../MNIST')
    imagesTest, labelsTest = mndata.load_training()
    imagesTrain, labelsTrain = mndata.load_testing()
    return imagesTrain, labelsTrain, imagesTest, labelsTest

In [130]:
def convolve(im, kernel):
    kernelSize = len(kernel)
    ret = (np.zeros((len(im) - 2*(kernelSize//2), len(im[0]) - 2*(kernelSize//2))))
    for i in range(kernelSize//2, len(im) - kernelSize//2):
        for j in range(kernelSize//2, len(im[0]) - kernelSize//2):
            for k in range(kernelSize):
                for l in range(kernelSize):
                    ret[i - kernelSize//2][j - kernelSize//2] += im[i - kernelSize//2 + k][j - kernelSize//2 + l]*kernel[k][l]
    return ret

In [131]:
def avgPool(im, k):
    ret = np.zeros(((len(im)//2), len(im[0])//2))
    for i in range(len(ret)):
        for j in range(len(ret[0])):
            for p in range(k):
                for l in range(k):
                    ret[i][j] += im[i*2 + p][j*2 + l]/(k**2)
    return ret

In [98]:
def reshape(x):
    return np.uint8(np.reshape(x, (int(math.sqrt(len(x))), int(math.sqrt(len(x))))))

In [106]:
def preprocess(im):
    return np.uint8(np.pad(im, (2, 2)))

In [112]:
xTrain, yTrain, xTest, yTest = loadMNIST()

In [113]:
for i in range(len(xTrain)):
    xTrain[i] = preprocess(reshape(xTrain[i]))
for i in range(len(xTest)):
    xTest[i] = preprocess(reshape(xTest[i]))

## Parameters
https://cdn.analyticsvidhya.com/wp-content/uploads/2021/03/Screenshot-from-2021-03-18-12-56-51.png